In [22]:
# CRM Customer Data Validation System

import pandas as pd
import re

Step 1: Load data

In [23]:
df = pd.read_csv("customer_data.csv", sep="\t")
print("Initial Shape:", df.shape)
df.head()

Initial Shape: (5, 5)


,CustomerID,Name,Email,Phone Number,City
0,1,Alice,alice@example.com,1234567890,New York
1,2,Bob,bob@example.com,987654321,Los Angeles
2,3,Charlie,charlie@example.com,1122334455,Chicago
3,4,David,david@example.com,5544332211,Houston
4,5,Eve,eve@example.com,9988776655,Phoenix


Step 2: Check for missing value

In [24]:
missing = df.isnull().sum()
print("Missing values:\n", missing[missing > 0])

Missing values:
 Series([], dtype: int64)


Step 3: Drop duplicates

In [25]:
duplicates = df.duplicated().sum()
print("Duplicates:", duplicates)
df_cleaned = df.drop_duplicates()

Duplicates: 0


Step 4: Validate email


In [27]:
def validate_email(email):
    pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"
    return bool(re.match(pattern, str(email)))

create dummy email column


In [32]:
if "Email" not in df_cleaned.columns:
    df_cleaned["Email"] = ["user{}@example.com".format(i) for i in range(len(df_cleaned))]

df_cleaned["Email_Valid"] = df_cleaned["Email"].apply(validate_email)
df.head()

,CustomerID,Name,Email,Phone Number,City
0,1,Alice,alice@example.com,1234567890,New York
1,2,Bob,bob@example.com,987654321,Los Angeles
2,3,Charlie,charlie@example.com,1122334455,Chicago
3,4,David,david@example.com,5544332211,Houston
4,5,Eve,eve@example.com,9988776655,Phoenix


Step 5: Validate phone number


In [33]:
df_cleaned["Phone Number"] = ["9876543210" if i % 2 == 0 else "abc" for i in range(len(df_cleaned))]
df_cleaned["Phone_Valid"] = df_cleaned["Phone Number"].astype(str).apply(lambda x: len(x) == 10 and x.isdigit())
df.head()

,CustomerID,Name,Email,Phone Number,City
0,1,Alice,alice@example.com,1234567890,New York
1,2,Bob,bob@example.com,987654321,Los Angeles
2,3,Charlie,charlie@example.com,1122334455,Chicago
3,4,David,david@example.com,5544332211,Houston
4,5,Eve,eve@example.com,9988776655,Phoenix


Step 6: Add record status


In [34]:
df_cleaned["Record_Status"] = df_cleaned.apply(
    lambda x: "Error" if not x["Email_Valid"] or not x["Phone_Valid"] else "Valid", axis=1
)

Step 7: Save cleaned data


In [36]:
df_cleaned.to_csv("cleaned_customer_data.csv", index=False)
df.head()

,CustomerID,Name,Email,Phone Number,City
0,1,Alice,alice@example.com,1234567890,New York
1,2,Bob,bob@example.com,987654321,Los Angeles
2,3,Charlie,charlie@example.com,1122334455,Chicago
3,4,David,david@example.com,5544332211,Houston
4,5,Eve,eve@example.com,9988776655,Phoenix


Step 8: summary report


In [37]:
errors = df_cleaned["Record_Status"].value_counts()
summary = {
    "Initial Records": len(df),
    "Duplicate Removed": duplicates,
    "Valid Records": int(errors.get("Valid", 0)),
    "Error Records": int(errors.get("Error", 0)),
    "Missing Values Fixed": df.isnull().sum().sum()
}

report_df = pd.DataFrame(list(summary.items()), columns=["Metric", "Value"])
report_df.to_excel("data_report.xlsx", index=False)

print("Data Cleaning Complete. Reports Generated.")

Data Cleaning Complete. Reports Generated.
